# AI Driven Drone Navigation
This project aims to create a navigation system with the help of AI for DJI Tello drone, with a single RGB camera and Inertial measurement unit (IMU) as sensors which may become
a challenging task to accomplish.

In [ ]:
import arcade
import copy
import cv2
from djitellopy import Tello
import numpy as np
import threading
import time

from depth_dataset_tool import DepthDatasetTool
from map import Map
from my_models import YOLOv5, MiDaS, AlexNet
from navigator import Navigator

In [2]:
# Ignore PyTorch user warning for cleaner output
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

## Initialise variables

In [3]:
tello = Tello()
tello.connect()
tello.streamon()

[INFO] tello.py - 122 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 437 - Send command: 'command'
[INFO] tello.py - 461 - Response command: 'ok'
[INFO] tello.py - 437 - Send command: 'streamon'
[INFO] tello.py - 461 - Response streamon: 'ok'


In [4]:
if tello.get_battery() <= 10:
    raise ValueError('DJI Tello battery is too low.')

In [7]:
env = Map((500,550))
env.center_window()
env.setup()

In [8]:
# Global variables
frame_read = tello.get_frame_read()
in_flight = False
auto_navigate = False
depth = None
obj = None

## Thread and functions

In [9]:
def drone_cam():
    while 1:
        # Camera capture
        img = frame_read.frame

        # battery_status = tello.get_battery()
        # cv2.putText(img, "Battery: {}%".format(battery_status), (5, 720 - 5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.imshow('Tello Camera', img)

        # Keyboard control
        key = cv2.waitKey(1) & 0xff
        if key == 27:  # ESC
            break

In [10]:
def keyboard_control():
    global in_flight, auto_navigate
    # Input window
    img = np.zeros((40, 300, 1), dtype = "uint8")
    while 1:
        cv2.imshow('Keyboard Control', img)

        # Keyboard control
        key = cv2.waitKey(1) & 0xff
        if key == 27:  # ESC
            break
        if key == ord('w'):
            # tello.move_forward(30)
            env.set_pos_direction('forward')
        elif key == ord('s'):
            tello.move_back(20)
        elif key == ord('a'):
            # tello.move_left(30)
            env.set_pos_direction('left')
        elif key == ord('d'):
            # tello.move_right(30)
            env.set_pos_direction('right')
        elif key == ord('D'):
            tello.rotate_clockwise(30)
        elif key == ord('A'):
            tello.rotate_counter_clockwise(30)
        elif key == ord('W'):
            tello.move_up(30)
        elif key == ord('S'):
            tello.move_down(30)
        elif key == ord('q'):
            if not auto_navigate:
                auto_navigate = True
            elif auto_navigate:
                auto_navigate = False
        elif key == ord('r'):
            env.setup()
        elif key == 32:  # Space
            if not in_flight:
                # Take-off drone
                tello.send_rc_control(0, 0, 0, 0)
                tello.takeoff()
                in_flight = True

            elif in_flight:
                # Land tello
                tello.send_rc_control(0, 0, 0, 0)
                # Handles dji tello random landing
                try:
                    tello.land()
                except:
                    pass
                in_flight = False

In [11]:
depth_model = MiDaS()
depth_model.init()
def depth_estimation():
    global depth
    while 1:
        img = frame_read.frame
        # Depth estimation
        depth = depth_model.predict(img)
        depth_norm = cv2.normalize(depth, None, 0, 1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_64F)
        cv2.imshow('Tello Depth Camera', depth_norm)
        key = cv2.waitKey(1) & 0xff
        if key == 27:  # ESC
            break
        time.sleep(0.05)

Using cache found in C:\Users\Eris/.cache\torch\hub\intel-isl_MiDaS_master
Using cache found in C:\Users\Eris/.cache\torch\hub\intel-isl_MiDaS_master


In [12]:
object_model = YOLOv5()
object_model.init()
def object_detection():
    global obj
    while 1:
        img = copy.copy(frame_read.frame)
        # Object detection
        obj = object_model.predict(img)
        cv2.imshow('Tello Object Camera', img)
        key = cv2.waitKey(1) & 0xff
        if key == 27:  # ESC
            break
        time.sleep(0.05)

Using cache found in C:\Users\Eris/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-4-4 torch 1.10.1+cu113 CUDA:0 (NVIDIA GeForce GTX 1080, 8192MiB)

Fusing layers... 
YOLOv5m6 summary: 378 layers, 35704908 parameters, 0 gradients, 50.0 GFLOPs
Adding AutoShape... 


In [13]:
depth_direction = AlexNet()
depth_direction.load()
def auto_navigation():
    global in_flight, auto_navigate
    navigator = Navigator()
    while 1:
        if in_flight and auto_navigate:
            lr, fb, ud, yaw = 0, 0, 0, 0
            env.set_speed(0.4)
            img = np.zeros((180, 1100, 1), dtype = "uint8")
            direction_str = None

            direction, near_obj_name, direction_value = navigator.get_direction(obj, depth_direction.predict(depth))
            if direction == 0:
                near_obj_name = None
                direction_str = 'forward'
                env.set_pos_direction(direction_str)
                fb = 20
            if direction == 1:
                direction_str = 'left'
                env.set_pos_direction(direction_str)
                lr = -20
            if direction == 2:
                direction_str = 'right'
                env.set_pos_direction(direction_str)
                lr = 20
            if direction == 3:
                direction_str = 'up'
                ud = 20
            if direction == 4:
                direction_str = 'down'
                ud = -20

            rot_direction = env.get_rot_direction()
            if rot_direction == 'left':
                yaw = -20 if env.get_target_angle_is_close() else -40
            if rot_direction == 'right':
                yaw = 20 if env.get_target_angle_is_close() else  40

            if env.get_red_light_status():
                lr, fb, ud = 0, 0, 0

            tello.send_rc_control(lr, fb, ud, yaw)

            if env.get_reach_goal():
                tello.send_rc_control(0, 0, 0, 0)
                tello.land()
                in_flight = False
                auto_navigate = False

            cv2.putText(img, 'Direction: {}'.format(direction_str), (25, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            cv2.putText(img, 'Rotation: {}'.format(rot_direction), (25, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            cv2.putText(img, 'Avoiding: {}'.format(near_obj_name), (25, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            cv2.putText(img, direction_value, (25, 160), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            cv2.imshow('Auto Navigation System', img)
            key = cv2.waitKey(1) & 0xff
            if key == 27:  # ESC
                break
        else:
            env.set_speed(0)

        angle = tello.get_yaw()
        if angle <= 0:
            open("E:\\GitHub\\Dissertation\\.temp\\rot", "w").write(str(-angle))
        else:
            open("E:\\GitHub\\Dissertation\\.temp\\rot", "w").write(str(360-angle))

In [14]:
def dataset_tool():
    ddt = DepthDatasetTool()
    new_depth_images = []
    new_depth_labels = []
    img = np.zeros((40, 300, 1), dtype="uint8")
    while 1:
        cv2.imshow('Dataset Collector', img)
        key = cv2.waitKey(1) & 0xff
        if key == 27:  # ESC
            break
        if key == ord('w'):
            new_depth_images.append(depth)
            new_depth_labels.append(0)
            print(len(new_depth_images))
        elif key == ord('s'):
            new_depth_images.append(depth)
            new_depth_labels.append(3)
            print(len(new_depth_images))
        elif key == ord('a'):
            new_depth_images.append(depth)
            new_depth_labels.append(1)
            print(len(new_depth_images))
        elif key == ord('d'):
            new_depth_images.append(depth)
            new_depth_labels.append(2)
            print(len(new_depth_images))
    ddt.collect(new_depth_images, new_depth_labels)

In [15]:
# threading.Thread(target=drone_cam).start()
threading.Thread(target=keyboard_control).start()
threading.Thread(target=depth_estimation).start()
threading.Thread(target=object_detection).start()
threading.Thread(target=auto_navigation).start()
# threading.Thread(target=dataset_tool).start()

## Main

In [ ]:
arcade.run()
cv2.destroyAllWindows()
if in_flight: tello.land()
tello.end()

Attempting to add texture: Images/finish-flag.png-0-0-0-0-False-False-False-Simple 
[2247219707184] No room for Images/finish-flag.png-0-0-0-0-False-False-False-Simple  size (501, 512)
[2247219707184] Resizing atlas from (512, 512) to (1024, 1024)
[2247219707184] Atlas resize took 0.008751400000001297 seconds
Attempting to add texture: Images/finish-flag.png-0-0-0-0-False-False-False-Simple 
Attempting to add texture: Images/drone.png-0-0-0-0-False-False-False-Simple 
[2247219707184] No room for Images/drone.png-0-0-0-0-False-False-False-Simple  size (720, 720)
[2247219707184] Resizing atlas from (1024, 1024) to (2048, 2048)
[2247219707184] Atlas resize took 0.006419699999998585 seconds
Attempting to add texture: Images/drone.png-0-0-0-0-False-False-False-Simple 
[INFO] tello.py - 470 - Send command (no response expected): 'rc 0 0 0 0'
Send command (no response expected): 'rc 0 0 0 0'
[INFO] tello.py - 437 - Send command: 'takeoff'
Send command: 'takeoff'
[INFO] tello.py - 461 - Respon

## Depth Direction Decision

In [1]:
# from dataset_collection import DepthDatasetTool
# ddt = DepthDatasetTool()
# ddt.editor()
# ddt.size()

In [ ]:
# depth_direction = AlexNet()
# depth_direction.train(numb_epoch=10)

In [ ]:
# i = 1
# img = np.load("depth_images.npy")
# depth_direction.predict(img[i])
# lab = np.load("depth_labels.npy")
# lab[i]

# Notes
Approx. 1px = 1cm
speed at 20 vel = 40cm/3s

### Order of movement
yaw (map)
lr & ud (obj/depth)
fb (obj/depth)

# To do list:

* Last testing phase
* Remove hard coded file directory
* Check and handles what if missing files before submitting to moodle
* Create Requirement.txt
* Documentation (if there's spare time)

# Known issues:

* Lack of dataset point.
Preferred: 60k rows
Acceptable: 25k rows
Reality: 5k rows

* Unable to detect left and right side of the drone. (Hardware)
* Inaccurate drone position data on Map. (Hardware & Light Condition)

# Things that can be improved:
* Remove unnecessary classes from a pretrained YOLOv5 model or train a new YOLOv5 model with a dataset that contains only necessary classes.
* Use other approaches such as semantic segmentation and orb slam.
* Gather more data points for training depth direction decision model.
* Create a multi-goal feature.